## Summary of Pre-processing ideas

1. use title to guess make
2. apply manufactured date and reg_date as average? Or maybe use clustering/nearest neighbours based on make/model?
3. potentially apply hierarchy to type_of_vehicle (convert it from nominal to ordinal)
4. add "almost new car" and "rare & exotic" from category column as separate features
5. estimate curb weight by mean for make and modelm otherwise mean for vehicle type
6. estimate power by mean for make and modelm otherwise mean for vehicle type
7. estimate engine cap by mean for make and modelm otherwise mean for vehicle type
8. Safe to assume that the missing fuel types are most likely petrol, since searching for petrol cars on sgcarmart returns a similarly larger proportion of cars as compared to searching for diesel cars
9. Only 176 missing no_of_owners, safe to assume 1 since all the cars are used cars, and 1 is the largest class
10. For depreciation, coe, dereg_value, road_tax, mileage, omv, arf, I am performing a random sampling from existing values in training to fill na values. The assumption is that the training data sample is representative of actual distribution



In [39]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# This will automatically reload src/dtree.py every time you make changes and save the file
%load_ext autoreload
%autoreload 2

%matplotlib inline

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
df_task1_train = pd.read_csv('train.csv')

df_task1_train.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price
0,1030324,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013.0,NaN,09-dec-2013,luxury sedan,"parf car, premium ad car, low mileage car",auto,1560.0,135.0,NaN,1997.0,1.0,17700.0,77100.0,1210.0,47514.0,73000.0,45330.0,50462.0,NaN,NaN,uncategorized,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",NaN,71300.0
1,1021510,Toyota Hiace 3.0M,NaN,hiace,high loan available! low mileage unit. wear an...,2014.0,NaN,26-jan-2015,van,premium ad car,manual,1740.0,NaN,diesel,2982.0,3.0,11630.0,10660.0,NaN,3648.0,110112.0,27502.0,1376.0,NaN,25-jan-2035,uncategorized,low mileage unit. well maintained vehicle. vie...,factory radio setting. front recording camera....,NaN,43800.0
2,1026909,Mercedes-Benz CLA-Class CLA180,mercedes-benz,cla180,1 owner c&c unit. full agent service with 1 mo...,2016.0,NaN,25-jul-2016,luxury sedan,"parf car, premium ad car",auto,1430.0,90.0,NaN,1595.0,1.0,15070.0,53694.0,740.0,44517.0,80000.0,27886.0,26041.0,NaN,NaN,uncategorized,responsive and fuel efficient 1.6l inline 4 cy...,dual electric/memory seats. factory fitted aud...,NaN,95500.0
3,1019371,Mercedes-Benz E-Class E180 Avantgarde,mercedes-benz,e180,"fully agent maintained, 3 years warranty 10 ye...",2019.0,NaN,17-nov-2020,luxury sedan,"parf car, almost new car, consignment car",auto,1635.0,115.0,NaN,1497.0,1.0,16400.0,40690.0,684.0,80301.0,9800.0,46412.0,56977.0,NaN,NaN,uncategorized,"1.5l inline-4 twin scroll turbocharged engine,...",64 colour ambient lighting. active parking ass...,NaN,197900.0
4,1031014,Honda Civic 1.6A VTi,NaN,civic,"kah motor unit! 1 owner, lowest 1.98% for full...",2019.0,NaN,20-sep-2019,mid-sized sedan,parf car,auto,1237.0,92.0,NaN,1597.0,1.0,10450.0,26667.0,742.0,36453.0,40000.0,20072.0,20101.0,NaN,NaN,uncategorized,"1.6l i-vtec engine, 123 bhp, earth dreams cvt ...","s/rims, premium leather seats, factory touch s...",NaN,103200.0


In [41]:
df_task1_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16784 entries, 0 to 16783
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16784 non-null  int64  
 1   title              16784 non-null  object 
 2   make               14624 non-null  object 
 3   model              16784 non-null  object 
 4   description        16439 non-null  object 
 5   manufactured       16590 non-null  float64
 6   original_reg_date  318 non-null    object 
 7   reg_date           16583 non-null  object 
 8   type_of_vehicle    16784 non-null  object 
 9   category           16784 non-null  object 
 10  transmission       16784 non-null  object 
 11  curb_weight        16205 non-null  float64
 12  power              14447 non-null  float64
 13  fuel_type          3490 non-null   object 
 14  engine_cap         16731 non-null  float64
 15  no_of_owners       16608 non-null  float64
 16  depreciation       163

In [42]:
df_task1_test = pd.read_csv('test.csv')

df_task1_test.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price
0,1029166,BMW X6 xDrive35i Sunroof (New 10-yr COE),bmw,x6,"owner consignment unit, viewing strictly by ap...",2012.0,NaN,27-jun-2012,suv,coe car,auto,2070.0,225.0,NaN,2979.0,3.0,14870.0,NaN,2362.0,NaN,122000.0,68994.0,68994.0,NaN,NaN,uncategorized,"smooth inline 6 3.0l turbo n55 engine, high sp...","20'' staggered m rims, carbon steering wheel, ...",NaN
1,1017714,Porsche 911 Carrera S Coupe 3.8A PDK (COE till...,porsche,911,the 911 carrera s displacing 3.8 litres with m...,2010.0,NaN,11-may-2010,sports car,"coe car, direct owner sale",auto,1530.0,283.0,NaN,3800.0,5.0,31920.0,31553.0,4366.0,23211.0,125000.0,115405.0,115405.0,NaN,NaN,uncategorized,responsive flat six 3.8 litres of naturally as...,"pasm, sports chrono, premium red leather packa...",NaN
2,1005265,Porsche Macan Diesel S 3.0A PDK,porsche,macan,comes with agent warranty till january 2022. a...,2016.0,NaN,18-jan-2017,suv,"parf car, premium ad car",auto,1955.0,190.0,diesel,2967.0,3.0,26640.0,53106.0,3336.0,102072.0,55965.0,69922.0,97860.0,NaN,NaN,uncategorized,black exterior with black interior. comes with...,"keyless entry/start, reverse camera, navi, ldw...",NaN
3,1029464,BMW 5 Series 530i Luxury,bmw,530i,a careful owner upgraded to a porsche macan. t...,2017.0,NaN,28-jun-2017,luxury sedan,"parf car, sgcarmart warranty cars",auto,1540.0,185.0,NaN,1998.0,1.0,19390.0,47501.0,1210.0,73330.0,100000.0,49363.0,61109.0,NaN,NaN,uncategorized,"2.0l bmw inline 4 cylinder twin power turbo, 2...","smart key, navigation, keyless entry, push sta...",NaN
4,1017727,Honda Vezel 1.5A X,NaN,vezel,100% non phv-unit! excellent condition and wel...,2016.0,NaN,05-jun-2017,suv,parf car,auto,1190.0,96.0,NaN,1496.0,2.0,9620.0,46489.0,682.0,34555.0,62000.0,20184.0,10258.0,NaN,NaN,uncategorized,"1.5l dohc i-vtec engine, 128 bhp, cvt auto tra...","sports rims, leather seats, andriod player, re...",NaN


In [43]:
df_task1_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         5000 non-null   int64  
 1   title              5000 non-null   object 
 2   make               4373 non-null   object 
 3   model              5000 non-null   object 
 4   description        4889 non-null   object 
 5   manufactured       4939 non-null   float64
 6   original_reg_date  100 non-null    object 
 7   reg_date           4939 non-null   object 
 8   type_of_vehicle    5000 non-null   object 
 9   category           5000 non-null   object 
 10  transmission       5000 non-null   object 
 11  curb_weight        4844 non-null   float64
 12  power              4325 non-null   float64
 13  fuel_type          1051 non-null   object 
 14  engine_cap         4985 non-null   float64
 15  no_of_owners       4944 non-null   float64
 16  depreciation       4868 

## All Preprocessing in one place!!

### Helper Methods

In [44]:
df_combined = df_task1_train.append(df_task1_test)
len(df_combined)

21784

In [45]:

def pop_std(x):
    return x.std(ddof=0)

curb_weight_dict_by_make_model = {}
curb_weight_dict_by_type = {}

def construct_curb_weight_dict():
    result_1 = df_combined.dropna(subset=['curb_weight']).groupby(['make', 'model'], as_index=False).agg({'curb_weight':['mean', pop_std]})
    result_1 = result_1.apply(pd.Series)
    result_1 = result_1.T.to_dict('list')

    result_2 = df_combined.dropna(subset=['curb_weight']).groupby(['type_of_vehicle'], as_index=False).agg({'curb_weight':['mean', pop_std]})
    result_2 = result_2.apply(pd.Series)
    result_2 = result_2.T.to_dict('list')

    for key in result_1:
        curb_weight_dict_by_make_model[result_1[key][0] + result_1[key][1]] = result_1[key][2]


    for key in result_2:
        curb_weight_dict_by_type[result_2[key][0]] = result_2[key][1]

construct_curb_weight_dict()

power_by_make_model = {}
power_by_type = {}

def construct_power_dict():
    result_1 = df_combined.dropna(subset=['power']).groupby(['make', 'model'], as_index=False).agg({'power':['mean', pop_std]})
    result_1 = result_1.apply(pd.Series)
    result_1 = result_1.T.to_dict('list')


    result_2 = df_combined.dropna(subset=['power']).groupby(['type_of_vehicle'], as_index=False).agg({'power':['mean', pop_std]})
    result_2 = result_2.apply(pd.Series)
    result_2 = result_2.T.to_dict('list')



    for key in result_1:
        power_by_make_model[result_1[key][0] + result_1[key][1]] = result_1[key][2]

    for key in result_2:
        power_by_type[result_2[key][0]] = result_2[key][1]

construct_power_dict()

engine_cap_by_make_model = {}
engine_cap_by_type = {}

def construct_engine_dict():
    result_1 = df_combined.dropna(subset=['engine_cap']).groupby(['make', 'model'], as_index=False).agg({'engine_cap':['mean', pop_std]})
    result_1 = result_1.apply(pd.Series)
    result_1 = result_1.T.to_dict('list')


    result_2 = df_combined.dropna(subset=['engine_cap']).groupby(['type_of_vehicle'], as_index=False).agg({'engine_cap':['mean', pop_std]})
    result_2 = result_2.apply(pd.Series)
    result_2 = result_2.T.to_dict('list')



    for key in result_1:
        engine_cap_by_make_model[result_1[key][0] + result_1[key][1]] = result_1[key][2]


    for key in result_2:
        engine_cap_by_type[result_2[key][0]] = result_2[key][1]

construct_engine_dict()

def get_manufactured_value(x):
    manufactured = df_combined['manufactured'].dropna().sample(n = 1)
    return manufactured.values[0]

def get_depreciation_value(x):
    depreciation = df_combined['depreciation'].dropna().sample(n = 1)
    return depreciation.values[0]

def get_coe_value(x):
    coe = df_combined['coe'].dropna().sample(n = 1)
    return coe.values[0]

def get_road_tax_value(x):
    road_tax = df_combined['road_tax'].dropna().sample(n = 1)
    return road_tax.values[0]

def get_dereg_value_value(x):
    dereg_value = df_combined['dereg_value'].dropna().sample(n = 1)
    return dereg_value.values[0]

def get_mileage_value(x):
    mileage = df_combined['mileage'].dropna().sample(n = 1)
    return mileage.values[0]

def get_omv_value(x):
    omv = df_combined['omv'].dropna().sample(n = 1)
    return omv.values[0]

def get_arf_value(x):
    arf = df_combined['arf'].dropna().sample(n = 1)
    return arf.values[0]

def get_engine_cap(x):
    if x['make'] + x['model'] in engine_cap_by_make_model:
        return engine_cap_by_make_model[x['make'] + x['model']]
    else:
        return engine_cap_by_type[x['type_of_vehicle']]

def get_power(x):
    if x['make'] + x['model'] in power_by_make_model:
        return power_by_make_model[x['make'] + x['model']]
    else:
        return power_by_type[x['type_of_vehicle']]

def get_curb_weight(x):
    if x['make'] + x['model'] in curb_weight_dict_by_make_model:
        return curb_weight_dict_by_make_model[x['make'] + x['model']]
    else:
        return curb_weight_dict_by_type[x['type_of_vehicle']]

In [46]:
# clean the original_reg_date and reg_date. if one is nan, take the value from other.
df_task1_test["original_reg_date"].fillna(df_task1_test["reg_date"], inplace=True)
df_task1_test["reg_date"].fillna(df_task1_test["original_reg_date"], inplace=True)

df_task1_train["original_reg_date"].fillna(df_task1_train["reg_date"], inplace=True)
df_task1_train["reg_date"].fillna(df_task1_train["original_reg_date"], inplace=True)


# convert from string to datetime
df_task1_test["reg_date"] = pd.to_datetime(df_task1_test["reg_date"])
df_task1_train["reg_date"] = pd.to_datetime(df_task1_train["reg_date"])
df_task1_test["original_reg_date"] = pd.to_datetime(df_task1_test["original_reg_date"])
df_task1_train["original_reg_date"] = pd.to_datetime(df_task1_train["original_reg_date"])

# fill in missing nan values for 'manufactured'. will take year from original_reg_date
df_task1_test["manufactured"].fillna(df_task1_test["original_reg_date"].dt.year, inplace=True)
df_task1_train["manufactured"].fillna(df_task1_train["original_reg_date"].dt.year, inplace=True)

df_task1_train["manufactured"] = df_task1_train["manufactured"].astype(int)
df_task1_test["manufactured"] = df_task1_test["manufactured"].astype(int)

In [47]:
df_task1_train['make'] = df_task1_train['make'].fillna(df_task1_train['title'].apply(lambda x: x.split()[0].lower()))

df_task1_train['curb_weight'] = df_task1_train['curb_weight'].fillna(df_task1_train.apply(lambda x: get_curb_weight(x), axis = 1))
df_task1_train['power'] = df_task1_train['power'].fillna(df_task1_train.apply(lambda x: get_power(x), axis = 1))
df_task1_train['engine_cap'] = df_task1_train['engine_cap'].fillna(df_task1_train.apply(lambda x: get_engine_cap(x), axis = 1))

df_task1_train['depreciation'] = df_task1_train['depreciation'].fillna(df_task1_train.apply(lambda x: get_depreciation_value(x), axis = 1))
df_task1_train['coe'] = df_task1_train['coe'].fillna(df_task1_train.apply(lambda x: get_coe_value(x), axis = 1))
df_task1_train['road_tax'] = df_task1_train['road_tax'].fillna(df_task1_train.apply(lambda x: get_road_tax_value(x), axis = 1))
df_task1_train['dereg_value'] = df_task1_train['dereg_value'].fillna(df_task1_train.apply(lambda x: get_dereg_value_value(x), axis = 1))
df_task1_train['mileage'] = df_task1_train['mileage'].fillna(df_task1_train.apply(lambda x: get_mileage_value(x), axis = 1))
df_task1_train['omv'] = df_task1_train['omv'].fillna(df_task1_train.apply(lambda x: get_omv_value(x), axis = 1))
df_task1_train['arf'] = df_task1_train['arf'].fillna(df_task1_train.apply(lambda x: get_arf_value(x), axis = 1))

# df_task1_train['depreciation'] = df_task1_train['depreciation'].fillna(int(df_combined['depreciation'].mean()))
# df_task1_train['coe'] = df_task1_train['coe'].fillna(int(df_combined['coe'].mean()))
# df_task1_train['road_tax'] = df_task1_train['road_tax'].fillna(int(df_combined['road_tax'].mean()))
# df_task1_train['dereg_value'] = df_task1_train['dereg_value'].fillna(int(df_combined['dereg_value'].mean()))
# df_task1_train['mileage'] = df_task1_train['mileage'].fillna(int(df_combined['mileage'].mean()))
# df_task1_train['omv'] = df_task1_train['omv'].fillna(int(df_combined['omv'].mean()))
# df_task1_train['arf'] = df_task1_train['arf'].fillna(int(df_combined['arf'].mean()))


df_task1_train['no_of_owners'] = df_task1_train['no_of_owners'].fillna(1)
df_task1_train['fuel_type'] = df_task1_train['fuel_type'].fillna('petrol')

In [48]:
# fill lifespan with original_reg_date

df_task1_test["lifespan"].fillna(df_task1_test["original_reg_date"], inplace=True)
df_task1_train["lifespan"].fillna(df_task1_train["original_reg_date"], inplace=True)

# convert to datetime
df_task1_test["lifespan"] = pd.to_datetime(df_task1_test["lifespan"])
df_task1_train["lifespan"] = pd.to_datetime(df_task1_train["lifespan"])

# add 20 years from original_reg_date to get lifespan; seems like those orig rows with lifespan is 20 years from reg date
df_task1_test['lifespan'] = df_task1_test['lifespan'] + pd.DateOffset(years=20)
df_task1_train['lifespan'] = df_task1_train['lifespan'] + pd.DateOffset(years=20)

In [49]:
df_task1_test['make'] = df_task1_test['make'].fillna(df_task1_test['title'].apply(lambda x: x.split()[0].lower()))

df_task1_test['curb_weight'] = df_task1_test['curb_weight'].fillna(df_task1_test.apply(lambda x: get_curb_weight(x), axis = 1))
df_task1_test['power'] = df_task1_test['power'].fillna(df_task1_test.apply(lambda x: get_power(x), axis = 1))
df_task1_test['engine_cap'] = df_task1_test['engine_cap'].fillna(df_task1_test.apply(lambda x: get_engine_cap(x), axis = 1))

df_task1_test['depreciation'] = df_task1_test['depreciation'].fillna(df_task1_test.apply(lambda x: get_depreciation_value(x), axis = 1))
df_task1_test['coe'] = df_task1_test['coe'].fillna(df_task1_test.apply(lambda x: get_coe_value(x), axis = 1))
df_task1_test['road_tax'] = df_task1_test['road_tax'].fillna(df_task1_test.apply(lambda x: get_road_tax_value(x), axis = 1))
df_task1_test['dereg_value'] = df_task1_test['dereg_value'].fillna(df_task1_test.apply(lambda x: get_dereg_value_value(x), axis = 1))
df_task1_test['mileage'] = df_task1_test['mileage'].fillna(df_task1_test.apply(lambda x: get_mileage_value(x), axis = 1))
df_task1_test['omv'] = df_task1_test['omv'].fillna(df_task1_test.apply(lambda x: get_omv_value(x), axis = 1))
df_task1_test['arf'] = df_task1_test['arf'].fillna(df_task1_test.apply(lambda x: get_arf_value(x), axis = 1))

# df_task1_test['depreciation'] = df_task1_test['depreciation'].fillna(int(df_combined['depreciation'].mean()))
# df_task1_test['coe'] = df_task1_test['coe'].fillna(int(df_combined['coe'].mean()))
# df_task1_test['road_tax'] = df_task1_test['road_tax'].fillna(int(df_combined['road_tax'].mean()))
# df_task1_test['dereg_value'] = df_task1_test['dereg_value'].fillna(int(df_combined['dereg_value'].mean()))
# df_task1_test['mileage'] = df_task1_test['mileage'].fillna(int(df_combined['mileage'].mean()))
# df_task1_test['omv'] = df_task1_test['omv'].fillna(int(df_combined['omv'].mean()))
# df_task1_test['arf'] = df_task1_test['arf'].fillna(int(df_combined['arf'].mean()))

df_task1_test['no_of_owners'] = df_task1_test['no_of_owners'].fillna(1)
df_task1_test['fuel_type'] = df_task1_test['fuel_type'].fillna('petrol')

In [50]:
df_task1_train.drop(['eco_category', 'indicative_price', 'listing_id'], axis=1, inplace=True, errors='ignore')
df_task1_train.head(n = 20)

,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,features,accessories,price
0,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013,2013-12-09,2013-12-09,luxury sedan,"parf car, premium ad car, low mileage car",auto,1560.0,135.0,petrol,1997.0,1.0,17700.0,77100.0,1210.0,47514.0,73000.0,45330.0,50462.0,NaN,2033-12-09,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",71300.0
1,Toyota Hiace 3.0M,toyota,hiace,high loan available! low mileage unit. wear an...,2014,2015-01-26,2015-01-26,van,premium ad car,manual,1740.0,111.0,diesel,2982.0,3.0,11630.0,10660.0,1039.0,3648.0,110112.0,27502.0,1376.0,NaN,2055-01-25,low mileage unit. well maintained vehicle. vie...,factory radio setting. front recording camera....,43800.0
2,Mercedes-Benz CLA-Class CLA180,mercedes-benz,cla180,1 owner c&c unit. full agent service with 1 mo...,2016,2016-07-25,2016-07-25,luxury sedan,"parf car, premium ad car",auto,1430.0,90.0,petrol,1595.0,1.0,15070.0,53694.0,740.0,44517.0,80000.0,27886.0,26041.0,NaN,2036-07-25,responsive and fuel efficient 1.6l inline 4 cy...,dual electric/memory seats. factory fitted aud...,95500.0
3,Mercedes-Benz E-Class E180 Avantgarde,mercedes-benz,e180,"fully agent maintained, 3 years warranty 10 ye...",2019,2020-11-17,2020-11-17,luxury sedan,"parf car, almost new car, consignment car",auto,1635.0,115.0,petrol,1497.0,1.0,16400.0,40690.0,684.0,80301.0,9800.0,46412.0,56977.0,NaN,2040-11-17,"1.5l inline-4 twin scroll turbocharged engine,...",64 colour ambient lighting. active parking ass...,197900.0
4,Honda Civic 1.6A VTi,honda,civic,"kah motor unit! 1 owner, lowest 1.98% for full...",2019,2019-09-20,2019-09-20,mid-sized sedan,parf car,auto,1237.0,92.0,petrol,1597.0,1.0,10450.0,26667.0,742.0,36453.0,40000.0,20072.0,20101.0,NaN,2039-09-20,"1.6l i-vtec engine, 123 bhp, earth dreams cvt ...","s/rims, premium leather seats, factory touch s...",103200.0
5,Mercedes-Benz A-Class A35 AMG 4MATIC Premium Plus,mercedes-benz,a35,one and only a35 with a45 features in sg! bidd...,2020,2020-12-25,2020-12-25,sports car,"imported used vehicle, coe car",auto,1559.5,225.0,petrol,1991.0,1.0,10080.0,10.0,1202.0,37726.0,12174.0,34000.0,8305.0,NaN,2040-12-25,powered by a newly developed 2 litre four cyli...,"fitted with amg aero package, amg performance ...",273800.0
6,Volvo V40 T2,volvo,v40,beautiful exterior and interior. straight chas...,2015,2015-12-11,2015-12-11,hatchback,"parf car, premium ad car",auto,1450.0,90.0,petrol,1498.0,3.0,11020.0,56001.0,684.0,37311.0,77777.0,22809.0,18933.0,NaN,2035-12-11,"1.5l inline 4 cylinder turbocharged engine, fr...","multi-function steering wheel, original audio ...",62500.0
7,Mercedes-Benz GLC-Class GLC63 S AMG 4MATIC+,mercedes-benz,amg,"100% loan available, lowest depre in market, 2...",2018,2018-06-25,2021-06-23,suv,"imported used vehicle, parf car, rare & exotic...",auto,2080.0,375.0,petrol,3982.0,1.0,8380.0,62100.0,3922.0,5847.0,31787.0,53386.0,68095.0,NaN,2038-06-25,"engine: 3982cc v8, dohc, 32v, twin turbo max p...","flagship of the glc range, the glc63 s shoehor...",367300.0
8,Toyota Corolla Altis 1.6A (COE till 04/2031),toyota,corolla,special 3 days promotion! only 2 ownership wit...,2011,2011-07-15,2011-07-15,mid-sized sedan,"coe car, premium ad car",auto,1195.0,90.0,petrol,1598.0,2.0,6280.0,42283.0,816.0,40951.0,20.0,15263.0,15263.0,NaN,2031-07-15,view specs of the toyota corolla altis,NaN,66900.0
9,BMW 3 Series 318i Sport,bmw,318i,super rare and beautiful black on black bmw pe...,2016,2016-10-27,2016-10-27,luxury sedan,parf car,auto,1425.0,100.0,petrol,1499.0,2.0,12640.0,56889.0,684.0,50999.0,111199.0,29860.0,28804.0,NaN,2036-10-27,full original bmw performance manufacturer spe...,bmw performance original sports accessories! v.

In [51]:
df_task1_test.drop(['eco_category', 'indicative_price', 'listing_id'], axis=1, inplace=True, errors='ignore')
df_task1_test.head(n = 20)

,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,features,accessories
0,BMW X6 xDrive35i Sunroof (New 10-yr COE),bmw,x6,"owner consignment unit, viewing strictly by ap...",2012,2012-06-27,2012-06-27,suv,coe car,auto,2070.0,225.0,petrol,2979.0,3.0,14870.0,25501.0,2362.0,56311.0,122000.0,68994.0,68994.0,NaN,2032-06-27,"smooth inline 6 3.0l turbo n55 engine, high sp...","20'' staggered m rims, carbon steering wheel, ..."
1,Porsche 911 Carrera S Coupe 3.8A PDK (COE till...,porsche,911,the 911 carrera s displacing 3.8 litres with m...,2010,2010-05-11,2010-05-11,sports car,"coe car, direct owner sale",auto,1530.0,283.0,petrol,3800.0,5.0,31920.0,31553.0,4366.0,23211.0,125000.0,115405.0,115405.0,NaN,2030-05-11,responsive flat six 3.8 litres of naturally as...,"pasm, sports chrono, premium red leather packa..."
2,Porsche Macan Diesel S 3.0A PDK,porsche,macan,comes with agent warranty till january 2022. a...,2016,2017-01-18,2017-01-18,suv,"parf car, premium ad car",auto,1955.0,190.0,diesel,2967.0,3.0,26640.0,53106.0,3336.0,102072.0,55965.0,69922.0,97860.0,NaN,2037-01-18,black exterior with black interior. comes with...,"keyless entry/start, reverse camera, navi, ldw..."
3,BMW 5 Series 530i Luxury,bmw,530i,a careful owner upgraded to a porsche macan. t...,2017,2017-06-28,2017-06-28,luxury sedan,"parf car, sgcarmart warranty cars",auto,1540.0,185.0,petrol,1998.0,1.0,19390.0,47501.0,1210.0,73330.0,100000.0,49363.0,61109.0,NaN,2037-06-28,"2.0l bmw inline 4 cylinder twin power turbo, 2...","smart key, navigation, keyless entry, push sta..."
4,Honda Vezel 1.5A X,honda,vezel,100% non phv-unit! excellent condition and wel...,2016,2017-06-05,2017-06-05,suv,parf car,auto,1190.0,96.0,petrol,1496.0,2.0,9620.0,46489.0,682.0,34555.0,62000.0,20184.0,10258.0,NaN,2037-06-05,"1.5l dohc i-vtec engine, 128 bhp, cvt auto tra...","sports rims, leather seats, andriod player, re..."
5,BMW Z4 sDrive28i,bmw,z4,stock z4. well maintained. nice interior and e...,2011,2012-03-29,2012-03-29,sports car,"parf car, direct owner sale",auto,1420.0,180.0,petrol,1997.0,5.0,76890.0,78000.0,1210.0,30305.0,76000.0,52268.0,52268.0,NaN,2032-03-29,view specs of the bmw z4,NaN
6,Toyota Corolla Axio 1.5A X (OPC),toyota,corolla,one owner! genuine and transparent! int rate f...,2015,2016-02-19,2016-02-19,mid-sized sedan,"opc car, parf car, low mileage car",auto,1090.0,80.0,petrol,1496.0,1.0,7970.0,26000.0,182.0,17890.0,40000.0,14082.0,9082.0,revised opc scheme . learn more about opc sche...,2036-02-19,view specs of the toyota corolla axio,"15"" sport rims, pioneer head unit, retractable..."
7,Mercedes-Benz A-Class A200 Sunroof,mercedes-benz,a200,well maintained in pristine condition. gorgeou...,2013,2013-05-23,2013-05-23,hatchback,parf car,auto,1395.0,115.0,petrol,1595.0,2.0,18680.0,64209.0,740.0,22384.0,36955.0,27420.0,20388.0,NaN,2033-05-23,"1.6l turbocharged inline 4 cylinders, 154bph, ...","factory fitted audio with bluetooth,media inte..."
8,Toyota Hiace 3.0M,toyota,hiace,accident-free unit. cheap and good 5 door toyo...,2018,2019-03-05,2019-03-05,van,premium ad car,manual,1700.0,111.0,diesel,2982.0,2.0,8760.0,22237.0,684.0,16699.0,28000.0,28136.0,1407.0,NaN,2059-03-04,view specs of the toyota hiace,NaN
9,Volkswagen Passat Variant 2.0A TSI R-Line,volkswagen,passat,2.0l turbo. 217bhp 350nm. reliable 6speed wet ...,2016,2016-05-26,2016-05-26,stationwagon,"parf car, direct owner sale",auto,1550.0,162.0,petrol,1984.0,2.0,16230.0,47000.0,1194.0,54051.0,65000.0,38176.0,45447.0,NaN,2036-05-26,panoramic roof. power tailgate. cruise control...,1 more free 90k servicing w ad. jus did 75k km...


#### One-hot Encoding

In [52]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder_make = OneHotEncoder(sparse=False)
one_hot_encoder_make.fit(df_combined['make'].values.reshape(-1, 1))

one_hot_encoder_model = OneHotEncoder(sparse=False)
one_hot_encoder_model.fit(df_combined['model'].values.reshape(-1, 1))

one_hot_encoder_type_of_vehicle = OneHotEncoder(sparse=False)
one_hot_encoder_type_of_vehicle.fit(df_combined['type_of_vehicle'].values.reshape(-1, 1))

one_hot_encoder_type_of_fuel_type = OneHotEncoder(sparse=False)
one_hot_encoder_type_of_fuel_type.fit(df_combined['fuel_type'].values.reshape(-1, 1))

one_hot_encoder_type_of_transmission = OneHotEncoder(sparse=False)
one_hot_encoder_type_of_transmission.fit(df_combined['transmission'].values.reshape(-1, 1))

df_task1_train = pd.concat((df_task1_train, pd.DataFrame(one_hot_encoder_make.transform(df_task1_train['make'].values.reshape(-1, 1)))), 1)
df_task1_train = pd.concat((df_task1_train, pd.DataFrame(one_hot_encoder_model.transform(df_task1_train['model'].values.reshape(-1, 1)))), 1)
df_task1_train = pd.concat((df_task1_train, pd.DataFrame(one_hot_encoder_type_of_vehicle.transform(df_task1_train['type_of_vehicle'].values.reshape(-1, 1)))), 1)
df_task1_train = pd.concat((df_task1_train, pd.DataFrame(one_hot_encoder_type_of_fuel_type.transform(df_task1_train['fuel_type'].values.reshape(-1, 1)))), 1)
df_task1_train = pd.concat((df_task1_train, pd.DataFrame(one_hot_encoder_type_of_transmission.transform(df_task1_train['transmission'].values.reshape(-1, 1)))), 1)

df_task1_train.head(n = 10)

,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,features,accessories,price,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,0,1,2,3,4,5,6,7,8,9,10,0,1,2,3,4,0,1
0,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013,2013-12-09,2013-12-09,luxury sedan,"parf car, premium ad car, low mileage car",auto,1560.0,135.0,petrol,1997.0,1.0,17700.0,77100.0,1210.0,47514.0,73000.0,45330.0,50462.0,NaN,2033-12-09,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",71300.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [53]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse=False)

df_task1_test = pd.concat((df_task1_test, pd.DataFrame(one_hot_encoder_make.transform(df_task1_test['make'].values.reshape(-1, 1)))), 1)
df_task1_test = pd.concat((df_task1_test, pd.DataFrame(one_hot_encoder_model.transform(df_task1_test['model'].values.reshape(-1, 1)))), 1)
df_task1_test = pd.concat((df_task1_test, pd.DataFrame(one_hot_encoder_type_of_vehicle.transform(df_task1_test['type_of_vehicle'].values.reshape(-1, 1)))), 1)
df_task1_test = pd.concat((df_task1_test, pd.DataFrame(one_hot_encoder_type_of_fuel_type.transform(df_task1_test['fuel_type'].values.reshape(-1, 1)))), 1)
df_task1_test = pd.concat((df_task1_test, pd.DataFrame(one_hot_encoder_type_of_transmission.transform(df_task1_test['transmission'].values.reshape(-1, 1)))), 1)

df_task1_test.head(n = 10)

,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,features,accessories,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,0,1,2,3,4,5,6,7,8,9,10,0,1,2,3,4,0,1
0,BMW X6 xDrive35i Sunroof (New 10-yr COE),bmw,x6,"owner consignment unit, viewing strictly by ap...",2012,2012-06-27,2012-06-27,suv,coe car,auto,2070.0,225.0,petrol,2979.0,3.0,14870.0,25501.0,2362.0,56311.0,122000.0,68994.0,68994.0,NaN,2032-06-27,"smooth inline 6 3.0l turbo n55 engine, high sp...","20'' staggered m rims, carbon steering wheel, ...",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### Drop columns that are already one-hot encoded

In [54]:
del df_task1_train['make']
del df_task1_train['model']
del df_task1_train['type_of_vehicle']
del df_task1_train['transmission']
del df_task1_train['fuel_type']

del df_task1_test['make']
del df_task1_test['model']
del df_task1_test['type_of_vehicle']
del df_task1_test['transmission']
del df_task1_test['fuel_type']


In [55]:

df_task1_train.info()
df_task1_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16784 entries, 0 to 16783
Columns: 868 entries, title to 1
dtypes: datetime64[ns](3), float64(858), int64(1), object(6)
memory usage: 111.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 867 entries, title to 1
dtypes: datetime64[ns](3), float64(857), int64(1), object(6)
memory usage: 33.1+ MB


In [56]:
df_task1_train['manufactured'] = df_task1_train['manufactured'].astype(int)
df_task1_train['original_reg_date'] = df_task1_train['original_reg_date'].astype(int)/ 10**9
df_task1_train['reg_date'] = df_task1_train['reg_date'].astype(int)/ 10**9
df_task1_train['lifespan'] = df_task1_train['lifespan'].astype(int)/ 10**9
df_task1_train.head()

,title,description,manufactured,original_reg_date,reg_date,category,curb_weight,power,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,features,accessories,price,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,0,1,2,3,4,5,6,7,8,9,10,0,1,2,3,4,0,1
0,BMW 3 Series 320i Gran Turismo M-Sport,1 owner! 320i gt m-sports model! big brake kit...,2013,1.386547e+09,1.386547e+09,"parf car, premium ad car, low mileage car",1560.0,135.0,1997.0,1.0,17700.0,77100.0,1210.0,47514.0,73000.0,45330.0,50462.0,NaN,2.017699e+09,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",71300.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [57]:
df_task1_test['manufactured'] = df_task1_test['manufactured'].astype(int)
df_task1_test['original_reg_date'] = df_task1_test['original_reg_date'].astype(int)/ 10**9
df_task1_test['reg_date'] = df_task1_test['reg_date'].astype(int)/ 10**9
df_task1_test['lifespan'] = df_task1_test['lifespan'].astype(int)/ 10**9
df_task1_test.head()

,title,description,manufactured,original_reg_date,reg_date,category,curb_weight,power,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,features,accessories,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,0,1,2,3,4,5,6,7,8,9,10,0,1,2,3,4,0,1
0,BMW X6 xDrive35i Sunroof (New 10-yr COE),"owner consignment unit, viewing strictly by ap...",2012,1.340755e+09,1.340755e+09,coe car,2070.0,225.0,2979.0,3.0,14870.0,25501.0,2362.0,56311.0,122000.0,68994.0,68994.0,NaN,1.971907e+09,"smooth inline 6 3.0l turbo n55 engine, high sp...","20'' staggered m rims, carbon steering wheel, ...",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

### Regression!!!

In [58]:
x_train = df_task1_train.drop(['price', 'title', 'description', 'category', 'features', 'accessories', 'opc_scheme'], axis=1, errors='ignore').to_numpy()

y_train = df_task1_train['price'].to_numpy()

x_test = df_task1_test.drop(['price', 'title', 'description', 'category', 'features', 'accessories', 'opc_scheme'], axis=1, errors='ignore').to_numpy()


### Add word vectors for titles (optional)

In [62]:
with open('train.npy', 'rb') as f:
    train_titles = np.load(f)
with open('test.npy', 'rb') as f:
    test_titles = np.load(f)

x_test = np.hstack((test_titles, x_test))
x_train = np.hstack((train_titles, x_train))
print(x_test.shape)
print(x_train.shape)

(5000, 1629)
(16784, 1629)


### MLP

In [65]:
from sklearn.neural_network import MLPRegressor
rgr_rforest = MLPRegressor(random_state=2, max_iter=500, verbose=1, tol = 0.00001).fit(x_train, y_train)

Iteration 1, loss = 367068879361909.43750000
Iteration 2, loss = 72305386000.74861145
Iteration 3, loss = 28124761365.52498245
Iteration 4, loss = 13449402213.73577118
Iteration 5, loss = 14955008762.87485123
Iteration 6, loss = 14599946428.52302933
Iteration 7, loss = 13879230449.43892097
Iteration 8, loss = 14843352758.83340263
Iteration 9, loss = 13212058946.15374374
Iteration 10, loss = 9889941696.15395164
Iteration 11, loss = 12820270520.12249374
Iteration 12, loss = 10839595494.37723732
Iteration 13, loss = 10532477234.25070763
Iteration 14, loss = 8629523675.90560532
Iteration 15, loss = 8417642952.60913086
Iteration 16, loss = 16935583477.08066750
Iteration 17, loss = 10950564072.72673607
Iteration 18, loss = 11017630559.55375099
Iteration 19, loss = 9496134777.52527237
Iteration 20, loss = 9700774475.86892700
Iteration 21, loss = 13284191576.85174751
Iteration 22, loss = 12955307545.96485710
Iteration 23, loss = 11221832875.60841942
Iteration 24, loss = 12334120182.81145668
It

### Random Forest

In [69]:
%%time

from sklearn.ensemble import RandomForestRegressor

rgr_rforest = RandomForestRegressor(verbose=1, n_jobs=5)
rgr_rforest.fit(x_train, y_train)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  2.7min


CPU times: user 25min 38s, sys: 16.2 s, total: 25min 54s
Wall time: 6min 22s


[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  6.4min finished


RandomForestRegressor(n_jobs=5, verbose=1)

In [70]:
import csv

y_pred = rgr_rforest.predict(x_test)

with open('y_pred.csv', 'w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    writer.writerow(['id','Predicted'])
    writer.writerows(enumerate(y_pred))

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished


### K-Nearest Neighbours

In [47]:
from sklearn import neighbors

knn = neighbors.KNeighborsRegressor(10, weights='distance')

y_test = knn.fit(x_train, y_train).predict(x_test)

[ 71300.  43800.  95500. 197900. 103200. 273800.  62500. 367300.  66900.
  87800.]
[ 71300.  43800.  95500. 197900. 103200. 273800.  62500. 367300.  66900.
  87800.]
